<a href="https://colab.research.google.com/github/djy2211/Credit/blob/main/sklearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

df = pd.read_csv('/content/train.csv')

In [ ]:
df = df.drop(index=28730)
df = df.reset_index(drop=True)

df

,ID,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수,대출등급
0,TRAIN_00000,12480000,36 months,6 years,RENT,72000000,18.90,15,부채 통합,0,0,0.0,0.0,0.0,C
1,TRAIN_00001,14400000,60 months,10+ years,MORTGAGE,130800000,22.33,21,주택 개선,0,373572,234060.0,0.0,0.0,B
2,TRAIN_00002,12000000,36 months,5 years,MORTGAGE,96000000,8.60,14,부채 통합,0,928644,151944.0,0.0,0.0,A
3,TRAIN_00003,14400000,36 months,8 years,MORTGAGE,132000000,15.09,15,부채 통합,0,325824,153108.0,0.0,0.0,C
4,TRAIN_00004,18000000,60 months,Unknown,RENT,71736000,25.39,19,주요 구매,0,228540,148956.0,0.0,0.0,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96288,TRAIN_96289,14400000,36 months,10+ years,MORTGAGE,210000000,9.33,33,신용 카드,0,974580,492168.0,0.0,0.0,C
96289,TRAIN_96290,28800000,60 months,10+ years,MORTGAGE,132000000,5.16,25,주택 개선,0,583728,855084.0,0.0,0.0,E
96290,TRAIN_96291,14400000,36 months,1 year,MORTGAGE,84000000,11.24,22,신용 카드,0,1489128,241236.0,0.0,0.0,A
96291,TRAIN_96292,15600000,36 months,5 years,MORTGAGE,66330000,17.30,21,부채 통합,2,1378368,818076.0,0.0,0.0,D


In [ ]:
df['주택소유상태'].unique()

array(['RENT', 'MORTGAGE', 'OWN'], dtype=object)

In [ ]:
df['대출기간'].unique()

array([' 36 months', ' 60 months'], dtype=object)

In [ ]:
# 36 months > 36, 60 months > 60
df['대출기간'] = df['대출기간'].str.extract('(\d+)').astype(int)

df['대출기간'].unique()

array([36, 60])

In [ ]:
df['근로기간'].unique()

array(['6 years', '10+ years', '5 years', '8 years', 'Unknown', '9 years',
       '2 years', '1 year', '3 years', '7 years', '4 years', '< 1 year',
       '10+years', '<1 year', '3', '1 years'], dtype=object)

In [ ]:
# DateTime 형태로 수정 10+ > 10년으로 고정 / 1년 보다 작은 경우 > 0으로, Unknown 도 0로 / 3은 일단 3년으로...> 모델확인 후 값이 좋지 않으면 0으로

Workdays_mapping = {'6 years' : 6, '10+ years' : 10, '5 years' : 5, '8 years' : 8, 'Unknown' : 0, '9 years': 9,
       '2 years' : 2, '1 year' : 1, '3 years' : 3, '7 years' : 7 , '4 years' : 4, '< 1 year' : 0,
       '10+years' : 10, '<1 year' : 0, '3' : 3, '1 years' : 1}

df['근로기간'] = df['근로기간'].map(Workdays_mapping)

df['근로기간'].unique()

array([ 6, 10,  5,  8,  0,  9,  2,  1,  3,  7,  4])

In [ ]:
df['대출목적'].unique()

array(['부채 통합', '주택 개선', '주요 구매', '휴가', '의료', '자동차', '신용 카드', '소규모 사업',
       '기타', '이사', '주택', '재생 에너지'], dtype=object)

In [ ]:
df = df.drop(columns=['연간소득'])

df

,ID,대출금액,대출기간,근로기간,주택소유상태,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수,대출등급
0,TRAIN_00000,12480000,36,6,RENT,18.90,15,부채 통합,0,0,0.0,0.0,0.0,C
1,TRAIN_00001,14400000,60,10,MORTGAGE,22.33,21,주택 개선,0,373572,234060.0,0.0,0.0,B
2,TRAIN_00002,12000000,36,5,MORTGAGE,8.60,14,부채 통합,0,928644,151944.0,0.0,0.0,A
3,TRAIN_00003,14400000,36,8,MORTGAGE,15.09,15,부채 통합,0,325824,153108.0,0.0,0.0,C
4,TRAIN_00004,18000000,60,0,RENT,25.39,19,주요 구매,0,228540,148956.0,0.0,0.0,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96288,TRAIN_96289,14400000,36,10,MORTGAGE,9.33,33,신용 카드,0,974580,492168.0,0.0,0.0,C
96289,TRAIN_96290,28800000,60,10,MORTGAGE,5.16,25,주택 개선,0,583728,855084.0,0.0,0.0,E
96290,TRAIN_96291,14400000,36,1,MORTGAGE,11.24,22,신용 카드,0,1489128,241236.0,0.0,0.0,A
96291,TRAIN_96292,15600000,36,5,MORTGAGE,17.30,21,부채 통합,2,1378368,818076.0,0.0,0.0,D


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [ ]:
encoder = OneHotEncoder(sparse=False)
df_encoded_onehot = pd.DataFrame(encoder.fit_transform(df[['주택소유상태', '대출목적']]),
                                columns=encoder.get_feature_names_out(['주택소유상태', '대출목적']))

label_encoder = LabelEncoder()
df_encoded_label = pd.DataFrame(label_encoder.fit_transform(df['대출등급']), columns=['대출등급'])

df_final = df.drop(['주택소유상태', '대출목적', '대출등급'],axis=1)

df_final = pd.concat([df_final, df_encoded_onehot, df_encoded_label], axis=1)
df_final

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,ID,대출금액,대출기간,근로기간,부채_대비_소득_비율,총계좌수,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,...,대출목적_신용 카드,대출목적_의료,대출목적_이사,대출목적_자동차,대출목적_재생 에너지,대출목적_주요 구매,대출목적_주택,대출목적_주택 개선,대출목적_휴가,대출등급
0,TRAIN_00000,12480000,36,6,18.90,15,0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
1,TRAIN_00001,14400000,60,10,22.33,21,0,373572,234060.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1
2,TRAIN_00002,12000000,36,5,8.60,14,0,928644,151944.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,TRAIN_00003,14400000,36,8,15.09,15,0,325824,153108.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
4,TRAIN_00004,18000000,60,0,25.39,19,0,228540,148956.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96288,TRAIN_96289,14400000,36,10,9.33,33,0,974580,492168.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
96289,TRAIN_96290,28800000,60,10,5.16,25,0,583728,855084.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4
96290,TRAIN_96291,14400000,36,1,11.24,22,0,1489128,241236.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
96291,TRAIN_96292,15600000,36,5,17.30,21,2,1378368,818076.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3


In [ ]:
X = df_final.drop(['ID', '대출등급'], axis=1)
y = df_final['대출등급']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("학습 데이터 크기:", X_train.shape, y_train.shape)
print("테스트 데이터 크기:", X_test.shape, y_test.shape)

학습 데이터 크기: (77034, 25) (77034,)
테스트 데이터 크기: (19259, 25) (19259,)


In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)
y_pred = dt_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'정확도: {accuracy:.2f}')
print('\n분류 보고서:\n', classification_report(y_test, y_pred))

정확도: 0.83

분류 보고서:
               precision    recall  f1-score   support

           0       0.87      0.88      0.87      3314
           1       0.86      0.86      0.86      5826
           2       0.85      0.86      0.86      5403
           3       0.78      0.76      0.77      2753
           4       0.75      0.72      0.74      1501
           5       0.68      0.70      0.69       385
           6       0.56      0.51      0.53        77

    accuracy                           0.83     19259
   macro avg       0.76      0.76      0.76     19259
weighted avg       0.83      0.83      0.83     19259



In [ ]:
test_df = pd.read_csv('/content/test.csv')
test_df

,ID,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수
0,TEST_00000,16800000,36 months,8 years,MORTGAGE,132000000,19.64,12,주택 개선,0,394692,146604.0,0.0,0.0
1,TEST_00001,8400000,36 months,5 years,RENT,89971200,15.84,25,부채 통합,0,0,0.0,0.0,0.0
2,TEST_00002,17280000,36 months,6 years,RENT,150000000,8.41,20,신용 카드,0,1786980,281820.0,0.0,0.0
3,TEST_00003,14400000,36 months,5 years,MORTGAGE,66000000,13.72,30,신용 카드,1,669024,281724.0,0.0,0.0
4,TEST_00004,27600000,36 months,5 years,RENT,55200000,30.50,12,신용 카드,0,1250052,614844.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64192,TEST_64192,30000000,36 months,3 years,MORTGAGE,78000000,22.08,27,부채 통합,2,1307532,763380.0,0.0,0.0
64193,TEST_64193,30000000,60 months,10+ years,MORTGAGE,109200000,12.06,26,부채 통합,0,960612,1245252.0,0.0,0.0
64194,TEST_64194,6120000,36 months,10+ years,RENT,39600000,28.80,33,부채 통합,0,131520,80880.0,0.0,0.0
64195,TEST_64195,11520000,36 months,10+ years,MORTGAGE,66000000,25.44,41,부채 통합,1,1339536,601872.0,0.0,0.0


In [ ]:
test_df['주택소유상태'].unique()

array(['MORTGAGE', 'RENT', 'OWN'], dtype=object)

In [ ]:
test_df['대출기간'].unique()

array([' 36 months', ' 60 months'], dtype=object)

In [ ]:
test_df['근로기간'].unique()

array(['8 years', '5 years', '6 years', 'Unknown', '10+ years', '3 years',
       '< 1 year', '1 year', '7 years', '4 years', '2 years', '9 years',
       '10+years', '<1 year', '3', '1 years'], dtype=object)

In [ ]:
test_df['대출목적'].unique()

array(['주택 개선', '부채 통합', '신용 카드', '기타', '주요 구매', '이사', '휴가', '소규모 사업',
       '주택', '의료', '자동차', '재생 에너지', '결혼'], dtype=object)

In [ ]:
test_df['대출기간'] = test_df['대출기간'].str.extract('(\d+)').astype(int)
test_df['근로기간'] = test_df['근로기간'].map(Workdays_mapping)
test_df = test_df.drop(columns=['연간소득'])

In [ ]:
test_df_encoded_onehot = pd.DataFrame(encoder.fit_transform(test_df[['주택소유상태', '대출목적']]),
                                columns=encoder.get_feature_names_out(['주택소유상태', '대출목적']))

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
test_df_final = test_df.drop(['주택소유상태', '대출목적'],axis=1)

test_df_final = pd.concat([test_df_final, test_df_encoded_onehot], axis=1)
test_df_final

,ID,대출금액,대출기간,근로기간,부채_대비_소득_비율,총계좌수,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,...,대출목적_소규모 사업,대출목적_신용 카드,대출목적_의료,대출목적_이사,대출목적_자동차,대출목적_재생 에너지,대출목적_주요 구매,대출목적_주택,대출목적_주택 개선,대출목적_휴가
0,TEST_00000,16800000,36,8,19.64,12,0,394692,146604.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,TEST_00001,8400000,36,5,15.84,25,0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,TEST_00002,17280000,36,6,8.41,20,0,1786980,281820.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,TEST_00003,14400000,36,5,13.72,30,1,669024,281724.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,TEST_00004,27600000,36,5,30.50,12,0,1250052,614844.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64192,TEST_64192,30000000,36,3,22.08,27,2,1307532,763380.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
64193,TEST_64193,30000000,60,10,12.06,26,0,960612,1245252.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
64194,TEST_64194,6120000,36,10,28.80,33,0,131520,80880.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
64195,TEST_64195,11520000,36,10,25.44,41,1,1339536,601872.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
test_df_final.columns

Index(['ID', '대출금액', '대출기간', '근로기간', '부채_대비_소득_비율', '총계좌수', '최근_2년간_연체_횟수',
       '총상환원금', '총상환이자', '총연체금액', '연체계좌수', '주택소유상태_MORTGAGE', '주택소유상태_OWN',
       '주택소유상태_RENT', '대출목적_결혼', '대출목적_기타', '대출목적_부채 통합', '대출목적_소규모 사업',
       '대출목적_신용 카드', '대출목적_의료', '대출목적_이사', '대출목적_자동차', '대출목적_재생 에너지',
       '대출목적_주요 구매', '대출목적_주택', '대출목적_주택 개선', '대출목적_휴가'],
      dtype='object')

In [ ]:
test_df_final = test_df_final.drop('대출목적_결혼', axis=1)

In [ ]:
test_df_final = test_df_final.drop(['ID'], axis=1)

In [ ]:
test_df_final_pred = dt_model.predict(test_df_final)

In [ ]:
test_df_final_pred

array([1, 1, 0, ..., 3, 2, 0])

In [ ]:
result = label_encoder.inverse_transform(test_df_final_pred)

In [ ]:
result

array(['B', 'B', 'A', ..., 'D', 'C', 'A'], dtype=object)

In [ ]:
test_df['ID']

0        TEST_00000
1        TEST_00001
2        TEST_00002
3        TEST_00003
4        TEST_00004
            ...    
64192    TEST_64192
64193    TEST_64193
64194    TEST_64194
64195    TEST_64195
64196    TEST_64196
Name: ID, Length: 64197, dtype: object

In [ ]:
dt_result = pd.read_csv('/content/sample_submission.csv')

In [ ]:
dt_result

,ID,대출등급
0,TEST_00000,G
1,TEST_00001,G
2,TEST_00002,G
3,TEST_00003,G
4,TEST_00004,G
...,...,...
64192,TEST_64192,G
64193,TEST_64193,G
64194,TEST_64194,G
64195,TEST_64195,G


In [ ]:
dt_result['대출등급'] = result

In [ ]:
dt_result

,ID,대출등급
0,TEST_00000,B
1,TEST_00001,B
2,TEST_00002,A
3,TEST_00003,C
4,TEST_00004,C
...,...,...
64192,TEST_64192,C
64193,TEST_64193,D
64194,TEST_64194,D
64195,TEST_64195,C


In [ ]:
dt_result.to_csv("submission.csv", index=False)
dt_result.to_csv("submission2.csv")